In [1]:
import pyspark
from jobs.core.base import JobHolder

tasks = JobHolder.get_registry().keys()
for name in tasks:
    print(name)

LoadCSV
GetTrainingData
DropNullAndDuplicateRow
DropNullColumns
SetTrainingData
BenchmarkModel


In [2]:
sc = pyspark.SparkContext('local[*]')

In [3]:
Task1 = JobHolder.get_registry()["LoadCSV"]
job1 = Task1(sc, filename="data_source/lending-club-loan-data/loan.csv.gz")
temp_table_1 = job1.execute()

In [4]:
Task2 = JobHolder.get_registry()["GetTrainingData"]
job2 = Task2(sc, previous_job_temp_table=temp_table_1)
temp_table_2 = job2.execute()

In [6]:
Task3 = JobHolder.get_registry()["DropNullAndDuplicateRow"]
job3 = Task3(sc, previous_job_temp_table=temp_table_2, na_threshold="0.7")
results3 = [res for res in job3.execute_extra()]
temp_table_3 = results3[0]

job3.get_metrics()

{'num_records': 483917,
 'num_columns': 139,
 'execution_time': 72.57651352882385}

In [7]:
Task4 = JobHolder.get_registry()["DropNullColumns"]
job4 = Task4(sc, previous_job_temp_table=temp_table_3, na_threshold="0.7")
results4 = [res for res in job4.execute_extra()]
temp_table_4 = results4[0]

job4.get_metrics()

{'num_records': 483917,
 'num_columns': 100,
 'columns_dropped': 39,
 'execution_time': 228.3570773601532}

In [57]:
Task5 = JobHolder.get_registry()["SetTrainingData"]
job5 = Task5(sc, previous_job_temp_table=temp_table_4)

results5 = [res for res in job5.execute_extra()]
temp_table_5 = results5[0]

job5.get_metrics()

{'num_records': 483916,
 'num_columns': 88,
 'execution_time': 1136.5675299167633}

In [55]:
Task6 = JobHolder.get_registry()["BenchmarkModel"]
job6 = Task6(sc, previous_job_temp_table=temp_table_5)
results6 = [res for res in job6.execute_extra()]
accuracy = results6[0]

In [56]:
print("Model accuracy")
print("{:.2f}%".format(float(accuracy) * 100))

Model accuracy
97.07%
